In [1]:
using Pipe
using DataFrames
# using StatsPlots
using CSV
# using RollingFunctions
using Query
# using GLMakie

using JSON
using DataFrames
using Dates
using Colors
# using ColorSchemes
using Indicators

import Colors
using Plots
using Interact
using Statistics
using StatsBase

include("utils.jl")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-507091004598753258\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-2132591645381257357\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-507091004598753258\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

calc_wavelet (generic function with 1 method)

In [2]:
json_data_folder = "../user_data/data/binance"
json_filename = "BTC_USDT-1m.json"
json_filepath = "$json_data_folder/$json_filename"
f = open(json_filepath, "r")
json_data = JSON.parse(f)
close(f)

dfa = DataFrame(unix_time = Int64[], open=Float64[], high=Float64[], low=Float64[], close=Float64[], volume=Float64[])
for r in json_data push!(dfa, r) end
dfa.unix_time .= dfa.unix_time ./1000
# dfa.time .= unix2datetime.(dfa.unix_time)

insertcols!(dfa, :time => unix2datetime.(dfa.unix_time))
nrow(dfa)

129177

In [3]:
start_date = DateTime(2021, 5, 1)
end_date = DateTime(2021, 5, 15)
vdf = dfa[(dfa.time .>= start_date) .& (dfa.time .<= end_date) , :]
ut, tt, hh, ll, cc = vdf.unix_time, vdf.time, vdf.high, vdf.low, vdf.close
length(tt)

20161

In [4]:
function backtest(stopsl, targetsl, lagLs )
#     println("$sl_slider, $lg_thresh_slider, $tp_slider")
    capital = 10000.
    qfee = 0.001
    risk_percent = 0.01
    

    dfo = DataFrame(ut = Int64[], 
            tt = DateTime[],
            order = Symbol[],
            oix = Int64[],
            reason = Symbol[],
            price = Float64[],
            stop = Float64[],
            qty = Float64[],
            pnl = Float64[],
            capital = Float64[]
        )

    push!(dfo, (ut = ut[1], tt=tt[1], oix =0, order = :long, reason = :toolong, price = 0, stop = 0,
                qty = 0, pnl = 0, capital = capital))


    in_long_pos = false 
    qlsellsig = 1.01
    qstop = 0.01
    in_i = 0
    for i in 300:length(cc)

        ord = last(dfo)
        if !in_long_pos
            if true &&
                cc[i] < lagLs[i-1]
                
#                 stop_price = cc[i] * (1 - sl_slider/100)
                stop_price = cc[i] * (1 - stopsl/100)

                risk_size = capital * risk_percent

                buy_price = cc[i]*(1+qfee)

                risk_price_range = (buy_price - stop_price)
                buyq = risk_size / risk_price_range


                push!(dfo, (ut = ut[i], tt=tt[i], oix=i, order = :long, reason = :enter, price = buy_price, qty = buyq,
                        stop = stop_price, pnl = 0, capital = capital))

                in_long_pos = true
            end
        else
#             sell = false
            if cc[i] > ord.price * (1 + targetsl/100)
                ord.stop = cc[i]
#             elseif i - ord.oix > negated_slider && all(cc[i-15:i] .< ema_2[i-15:i]) && ord.stop < ord.price
#                 ord.stop = ema_2[i]
#             elseif cc[i]
#             elseif cc[i-1] < ema_2[i-1] && cc[i] > ema_2[i] && ord.stop < ord.price
#                 ord.stop = ord.price
#             elseif minimum(cc[i-30:i]) > ord.stop
#                 ord
#             elseif minimum(cc[i-26*2]) < minimum(cc[i-26*1]) && 
#                 cc[i] > 1.005 * minimum(cc[i-26*2]) &&
#                 ord.stop < minimum(cc[i-26*2])
                
#                 ord.stop = minimum(cc[i-26*2])
                
            end
            
            if cc[i] <= ord.stop
                sell_price = cc[i]*(1-qfee)
                profit_loss = ord.qty * (sell_price - ord.price)
                capital = capital + profit_loss

                push!(dfo, (ut = ut[i], tt=tt[i], oix=i, order = :long, reason = :exit, price = sell_price, stop = 0, 
                         qty = ord.qty, pnl = profit_loss, capital = capital))

                in_long_pos = false
            end
           

        end
    #     println(capital)

    end
    CSV.write("trades.csv", dfo)
    dfo
end

backtest (generic function with 1 method)

In [14]:
arlen = length(cc)
wsl = window_slider(arlen, subs=2^2); wsl[:log2_slider] = 13;
wst = Interact.@map first(&wsl)
wed = Interact.@map last(&wsl)
plot_box = Observable{Any}(dom"div"())
rng = Interact.@map &wst: &wed
wcc = Interact.@map cc[&rng]
wll = Interact.@map cc[&rng]

wut = Interact.@map ut[&rng]
# wtt = Interact.@map tt[&rng]

lagLsl = slider(1:20:1500; label="lagL", default=14); lagLsl[] = 300;
lagLs = Interact.@map  [fill(ll[1], &lagLsl)..., [minimum(ll[i-&lagLsl:i]) for i in &lagLsl+1:arlen]...]

# lagHsl = slider(1:20:1200; label="lagH", default=14); lagHsl[] = 600;
# lagHs = Interact.@map  [fill(hh[1], &lagHsl)..., [maximum(hh[i-&lagHsl:i]) for i in &lagHsl+1:arlen]...]

wlagLs = Interact.@map (&lagLs)[&rng]
# wlagHs = Interact.@map (&lagHs)[&rng]

stopsl = slider(0:0.1:5; label="stop", default=14); stopsl[] = 2;
targetsl = slider(0:0.1:10; label="stop", default=14); targetsl[] = 2;

# &negated_slider, &cooloff_slider, &ema_1_trans, &ema_1_ch, &ema_2_trans, &ema_2_ch
# dfo = Interact.@map backtest(&stopsl, &targetsl, &lagLs )

# wdfo = Interact.@map filter(row -> (row.ut >= ut[&wst] && row.ut <= ut[&wed]), &dfo, view=false)

# res_capital = Interact.@map trunc(last(&wdfo).capital)

# buys = Interact.@map filter(row -> row.reason == :enter, &wdfo)
# losses = Interact.@map filter(row -> row.reason == :exit && row.pnl < 0, &wdfo)
# profits = Interact.@map filter(row -> row.reason == :exit && row.pnl > 0, &wdfo)
# no_losses = Interact.@map nrow(&losses)
# no_profits = Interact.@map nrow(&profits)
# tot_losses = Interact.@map Int(trunc(sum((&losses).pnl)))
# tot_profits = Interact.@map Int(trunc(sum((&profits).pnl)))


# wbuys = Interact.@map filter(row -> row.reason == :enter, &wdfo)
# wlosses = Interact.@map filter(row -> row.reason == :exit && row.pnl < 0, &wdfo)
# wprofits = Interact.@map filter(row -> row.reason == :exit && row.pnl > 0, &wdfo)
# , wlagLs, wlagHs, wbuys, wlosses, wprofits
function draw_it_all(wut, wcc, wll, wlagLs)    
    f2 = plot(wcc, size=(1500,600))
    plot!(wll, lw=0.2)
    plot!(wlagLs)
#     plot!(wlagHs)
    plot!(wlagLs .* 0.995, fillrange=wlagLs*1.005, lw=0, fillalpha=0.15)
    
    
#     nrow(wbuys) > 0 && vline!(wbuys.ut,lw=0.5, color=:blue)
#     nrow(wprofits) > 0 && vline!(wprofits.ut,lw=0.5, color=:green)
#     nrow(wlosses) > 0 && vline!(wlosses.ut,lw=0.5, color=:red)
#     plot!(wlag2mins)
    dom"div"(f2) 
end
# , wbuys, wlosses, wprofits
map!(draw_it_all, plot_box, wut, wcc, wll, wlagLs)
ui = dom"div"(
        hbox(lagLsl,lagHsl,stopsl),
        plot_box,wsl
    )
# wsl[:log2_slider][] = 11
# lagsl[] = 14
ui





(div
  (div { style=Dict("display" => "flex", "flex-direction" => "row") }
    Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["lagL"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 75, :default => 14, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :min => 1, :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Any} with 2 listeners. Value:
16, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000111f12210, Task (runnable) @0x0000000111f12210), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"21\",\"41\",\"61\",\"81\",\"101\",\"121\",\"141\",\"161\",\"181\",\"201\",\"221\",\"241\",\"261\",\"281\",\"301\",\"321\",\"341\",\"361\",\"381\",\"401\",\"421\",\"441\",\"461\",\"481\",\"501\",\

In [6]:
# wbuys[]

In [7]:
# using WebIO
# WebIO.install_jupyter_nbextension()



In [8]:
# WebIO.install_jupyter_labextension()